In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
import re
import ssl

In [ ]:
# Source: https://github.com/gunthercox/ChatterBot/issues/930#issuecomment-322111087 
# This is to fix the SSL error when downloading nltk data, which is a known issue on Mac OS
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

In [ ]:
# Function for text preprocessing
def preprocess_text(text):
    print(text)

    # Remove digits
    text = text.replace('\d+', '')
    
    # Clean urls
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    text = url_pattern.sub('', text)
    
    # Lowercasing
    text = text.lower()

    # Tokenization
    tokens = word_tokenize(text)

    # Removing stopwords and non-alphanumeric characters
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.isalnum() and token not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return ' '.join(tokens)

In [ ]:
def remove_punctuation(words):
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', (word))
        new_words.append(new_word)
    return new_words

In [ ]:
train_df = pd.read_csv("data_train_preprocessed.csv")
test_df = pd.read_csv("data_test_preprocessed.csv")

train_df['processed_text_alt'] = train_df['processed_text_alt'].fillna('')
test_df['processed_text_alt'] = test_df['processed_text_alt'].fillna('')

print(train_df['processed_text_alt'])

In [ ]:
train_tokens = [token for tokens_list in train_df['processed_text_alt'] for token in tokens_list]
preprocessed_train_tokens = [token for tokens_list in train_df['processed_text_alt'] for token in tokens_list]

vocabulary_size = len(set(train_tokens))
vocabulary_size_preprocessed = len(set(preprocessed_train_tokens))

print(f'Vocabulary Size Preprocessed: {vocabulary_size_preprocessed}')
print(f'Vocabulary Size: {vocabulary_size}')

# Split the dataset into training and validation sets
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [ ]:
# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=3000)  # Adjust max_features based on dataset size
X_train = vectorizer.fit_transform(train_df['processed_text_alt']).toarray()
X_val = vectorizer.transform(val_df['processed_text_alt']).toarray()
X_test = vectorizer.transform(test_df['processed_text_alt']).toarray()
y_train = (train_df['binary_labels']).astype(int).values
y_val = (val_df['binary_labels']).astype(int).values
y_test = (test_df['binary_labels']).astype(int).values

In [ ]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [ ]:
# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [ ]:
from wordcloud import WordCloud
#Frequency of words
fdist = FreqDist(train_df['processed_text_alt'])
#WordCloud
wc = WordCloud(width=800, height=400, max_words=50).generate_from_frequencies(fdist)
plt.figure(figsize=(12,10))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
# Define the Neural Network model
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

In [ ]:
class DeepNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.layer1 = nn.Linear(input_size, 2000)
        self.act1 = nn.ReLU()
        self.layer2 = nn.Linear(2000, 500)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(500, 100)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(100, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.sigmoid(self.output(x))
        return x

In [ ]:
# Instantiate the model
input_size = X_train.shape[1]
print ("Input size:", input_size)
hidden_size = 500
output_size = 1  # Binary classification
model = SimpleNN(input_size, hidden_size, output_size)

In [ ]:
# Loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training loop with validation
epochs = 10

train_losses = []
val_losses = []

avg_train_losses = []
avg_val_losses = []

for epoch in range(epochs):
    
    model.train()
    
    for batch_X, batch_y in train_loader:
        
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs.squeeze(), batch_y)
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())
        # print(f'Epoch: {epoch + 1}, Batch Loss: {loss.item()}')
        

    # Validation
    model.eval()
    
    with torch.no_grad():
        
        val_outputs = model(X_val_tensor)
        val_loss = criterion(val_outputs.squeeze(), y_val_tensor)
        val_losses.append(loss.item())

    # Print average training loss for the epoch
    avg_train_loss = sum(train_losses) / len(train_losses)
    avg_train_losses.append(avg_train_loss)
    print(f'Epoch: {epoch + 1}, Average Training Loss: {avg_train_loss}')

    avg_val_loss = sum(val_losses) / len(val_losses)
    avg_val_losses.append(avg_val_loss)
    print(f'Average Validation Loss: {avg_val_loss}')

In [ ]:
import seaborn as sns
import numpy as np

# plot results
def plot_results():
    # styling from seaborn.
    sns.set(style='darkgrid')
    # uncrease the plot size and font size.
    sns.set(font_scale=1.5)
    plt.rcParams["figure.figsize"] = (12,6)

    # plot the learning curve.
    plt.plot(np.arange(1,epochs+1), avg_train_losses, 'b-o', label="Training")
    plt.plot(np.arange(1,epochs+1), avg_val_losses, 'g-o', label="Validation")

    # Label the plot.
    plt.title("Training & Validation Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.xticks(list(range(1, epochs+1)))

    return plt.show()


plot_results()

In [ ]:
# Evaluation on the test set
model.eval()
with torch.no_grad():
    y_pred_test = model(X_test_tensor).squeeze().numpy()

y_pred_test_binary = (y_pred_test > 0.5).astype(int)

test_accuracy = accuracy_score(y_test, y_pred_test_binary)

print(f"Testing Accuracy: {test_accuracy}")

# Classification Report
print("Classification Report on Test Set:")
# TODO: Fix zero_division error, the precision result for hate (class 1) is not right currently
print(classification_report(y_test, y_pred_test_binary))